In [1]:
import numpy as np
import cv2 as cv
from PIL import Image
from collections import defaultdict
from pathlib import Path
from contextlib import contextmanager
from deep_cluster.dataloader import LandmarkDataset, SequenceDataset
from matplotlib import pyplot as plt
from matplotlib import cm
import os
from collections import Counter
import torch
from torch.utils.data import ConcatDataset
import pickle
import re
data_root = Path("/home/orel/Storage/Data/K6/")
plt.rcParams['svg.fonttype'] = 'none'
landmark_file = data_root/'2020-03-23'/'Down'/'0008DeepCut_resnet50_Down2May25shuffle1_1030000.h5'
video_file = data_root/'2020-03-23'/'Down'/'0008.MP4'

In [3]:
# root directory of data
data_root = Path("/home/orel/Storage/Data/K7/")
landmark_files = []
for subdir in os.listdir(data_root):
    if not os.path.exists(data_root/subdir/'Down'):
        continue
    print(subdir)
    for file in os.listdir(data_root/subdir/'Down'):
        if re.match(r"00\d*DeepCut.*_1030000\.h5", file):
            lfile = data_root/subdir/'Down'/file
            landmark_files.append(lfile)

2020-08-10
2020-08-04
2020-08-16
2020-08-12
2020-08-13
2020-08-06
2020-08-05


In [9]:
[cm.tab20(i) for i in range(20)]
cm.tab20.colors

((0.12156862745098039, 0.4666666666666667, 0.7058823529411765),
 (0.6823529411764706, 0.7803921568627451, 0.9098039215686274),
 (1.0, 0.4980392156862745, 0.054901960784313725),
 (1.0, 0.7333333333333333, 0.47058823529411764),
 (0.17254901960784313, 0.6274509803921569, 0.17254901960784313),
 (0.596078431372549, 0.8745098039215686, 0.5411764705882353),
 (0.8392156862745098, 0.15294117647058825, 0.1568627450980392),
 (1.0, 0.596078431372549, 0.5882352941176471),
 (0.5803921568627451, 0.403921568627451, 0.7411764705882353),
 (0.7725490196078432, 0.6901960784313725, 0.8352941176470589),
 (0.5490196078431373, 0.33725490196078434, 0.29411764705882354),
 (0.7686274509803922, 0.611764705882353, 0.5803921568627451),
 (0.8901960784313725, 0.4666666666666667, 0.7607843137254902),
 (0.9686274509803922, 0.7137254901960784, 0.8235294117647058),
 (0.4980392156862745, 0.4980392156862745, 0.4980392156862745),
 (0.7803921568627451, 0.7803921568627451, 0.7803921568627451),
 (0.7372549019607844, 0.74117647

In [ ]:

from time import time
from contextlib import contextmanager

@contextmanager
def timethis(label):
    t0 = time()
    yield
    elapsed = time() - t0
    print(f"{label} took {elapsed} seconds")

In [6]:
from landmarks_video import Video, LandmarksVideo

video = Video(video_file)
vid = LandmarksVideo(landmark_file.parent)
vid1 = LandmarksVideo(landmark_files[-1].parent)

In [26]:
vid.landmarks.body_parts, vid1.landmarks.body_parts

body_parts = list(vid1.landmarks.body_parts)
body_parts.index('tailbase')

16

In [ ]:
model_dir = Path('../models/11_03')

with open(model_dir / 'labels_dict.pkl', 'rb') as file:
    labels_dict = pickle.load(file)
    
with open(model_dir / 'data_dict.pkl', 'rb') as file:
    data_dict = pickle.load(file)
    
with open(model_dir / 'segments_dict.pkl', 'rb') as file:
    segments_dict = pickle.load(file)
    
with open(model_dir / 'x_encoded_dict.pkl', 'rb') as file:
    X_encoded_dict = pickle.load(file)

In [ ]:
vid = LandmarksVideo(landmark_file.parent)

segments = segments_dict[landmark_file]
labels = labels_dict[landmark_file]
x_encoded = X_encoded_dict[landmark_file]
labels = labels_dict[landmark_file]
n_clusters = max(labels) + 1


In [ ]:
from dataclasses import dataclass, field
import typing
from collections import namedtuple
get_segment_mean = lambda seg: x_encoded[seg[1]//4: seg[1]//4  + seg[2]//4].mean(axis=1)

@dataclass
class Segment:
    cluster_id : int
    start_frame: int
    n_frames: int
    encoded_mean: typing.Any = field(init=False)
    
    def __post_init__(self):
        self.encoded_mean = x_encoded[self.start_frame//4: self.start_frame//4  + self.n_frames//4].mean(axis=0)
        
segments = list(map(lambda seg: Segment(*seg), segments_dict[landmark_file]))     
segments_by_cluster = {cluster:[seg for seg in segments if seg.cluster_id==cluster] for cluster in range(n_clusters)}
segments_by_cluster = {k: v for k, v in segments_by_cluster.items() if len(v) > 5}
print({k: len(segs) for k, segs in segments_by_cluster.items()})
print({k: np.mean([seg.n_frames for seg in segs]) for k, segs in segments_by_cluster.items()})

In [ ]:
segment_encs = np.stack([seg.encoded_mean for seg in segments])
segment_cluster_ids = np.array([seg.cluster_id for seg in segments])
dists = np.linalg.norm(segment_encs[...,np.newaxis] - segment_encs.T[np.newaxis], axis=1)
nearest_segment_ids = np.argsort(dists, axis=1)
plt.plot((segment_cluster_ids[nearest_segment_ids] == segment_cluster_ids[:,np.newaxis]).mean(axis=0))

In [ ]:
def get_pos_neg(anchor_id, encoded=x_encoded, labels=labels, min_dist=9.0, max_dist=10.0):
    anchor = encoded[anchor_id]
    dists = np.linalg.norm(encoded - anchor, axis=1)
    in_dist = np.where(np.logical_and(min_dist < dists, dists < max_dist))[0]
    positives = in_dist[labels[in_dist] == labels[anchor_id]]
    negatives = in_dist[labels[in_dist] != labels[anchor_id]]
    pos_id = np.random.choice(positives)
    neg_id = np.random.choice(negatives)
    return pos_id, neg_id


def sample_triplet(encoded=x_encoded, labels=labels):
    anchor_id = np.random.choice(len(encoded))
    anchor = encoded[anchor_id]
    dists = np.linalg.norm(encoded - anchor, axis=-1)
    nums, bins, *_ = plt.hist(dists, bins=100)
    plt.close()
    to_id = np.where(np.cumsum(nums) / sum(nums) > 0.9)[0][0]
    bins = bins[:to_id]
    bin_idxs = [np.where(np.logical_and(bins[:-1, np.newaxis] < dists[np.newaxis], dists[np.newaxis] < bins[1:, np.newaxis])[i])[0] for i in range(len(bins)-1)]
    n_same_cluster_in_bin = np.array([(labels[bin_idxs[i]] == labels[anchor_id]).sum() for i in range(len(bins) - 1)])
    n_other_clusters_in_bin = np.array([(labels[bin_idxs[i]] != labels[anchor_id]).sum() for i in range(len(bins) - 1)])
    ratio = n_same_cluster_in_bin / (n_same_cluster_in_bin + n_other_clusters_in_bin + 1e-6)
    try:
        min_bin_idx = np.where(ratio > 0.5)[0][-1]
    except IndexError:
        min_bin_idx = 0
    max_bin_idx = np.where(np.logical_and(ratio < 0.5, np.arange(len(bins) - 1) >= min_bin_idx))[0][0]
    min_dist, max_dist = bins[min_bin_idx], bins[max_bin_idx+1]
    assert min_dist <= max_dist, (min_dist, max_dist)
    assert np.sum(nums[min_bin_idx: max_bin_idx+1]) > 0
    print(ratio[min_bin_idx:max_bin_idx + 1].mean())
    in_dist = np.where(np.logical_and(min_dist < dists, dists < max_dist))[0]
    if len(in_dist) < 5:
        raise Exception('not enough samples in distance range')
    positives = in_dist[labels[in_dist] == labels[anchor_id]]
    negatives = in_dist[labels[in_dist] != labels[anchor_id]]
    pos_id = np.random.choice(positives)
    neg_id = np.random.choice(negatives)
    return anchor_id, pos_id, neg_id

def triplets_gen(n_triplets=100, encoded=x_encoded, labels=labels):
    i = 0
    while True:
        try:
            yield sample_triplet(encoded, labels)
        except Exception:
            continue
        i += 1
        if i == n_triplets:
            break

            
def triplets_segments_gen(n_triplets=100, segments=segments):
    segment_encs = np.stack([seg.encoded_mean for seg in segments])
    segment_cluster_ids = np.array([seg.cluster_id for seg in segments])
    i = 0
    while True:
        try:
            anchor_id, pos_id, neg_id = sample_triplet(encoded=segment_encs, labels=segment_cluster_ids)
            anchor, pos, neg = segments[anchor_id], segments[pos_id], segments[neg_id]
            if min([seg.n_frames for seg in [anchor, pos, neg]]) <= 20:
                continue
            if max([seg.n_frames for seg in [anchor, pos, neg]]) > 480:
                continue
            if max([seg.n_frames for seg in [anchor, pos, neg]]) / min([seg.n_frames for seg in [anchor, pos, neg]]) > 3:
                continue
            yield anchor, pos, neg
        except Exception:
            continue
        i += 1
        if i == n_triplets:
            break

%pdb on
for tri in triplets_segments_gen(10, segments):
    print(tri)

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

from matplotlib import animation, rc
from IPython.display import HTML

In [ ]:
import math
def get_seg_clip(seg, n_frames, fps=120):
    frames = vid[seg.start_frame: seg.start_frame + seg.n_frames: 240//fps]
    frames = list(frames)
    if n_frames < len(frames):
        n_frames_to_discard = len(frames) - n_frames
        n_frames_to_discard_beginning = math.floor(n_frames_to_discard / 2)
        n_frames_to_discard_end = math.ceil(n_frames_to_discard / 2)
        frames = frames[n_frames_to_discard_beginning: - n_frames_to_discard_end]
    n_pad = n_frames - len(frames)
    pad_beginning, pad_ending = math.floor(n_pad / 2), math.ceil(n_pad / 2)
    frames = [frames[0]] * pad_beginning + frames + [frames[-1]] * pad_ending
    return np.stack(frames)

def animate_fn(clips, artists):
    def animate(i):
        for im, clip in zip(artists, clips):
            im.set_data(clip[i])
        return artists
    return animate

def show_clips(clips, interval=24):
    fig, axes = plt.subplots(ncols=len(clips), figsize=(12, 6))
    n_frames = min(len(clip) for clip in clips)
    for i, ax in enumerate(axes):
        ax.set_title(f"clip {i}")
    artists = [ax.imshow(np.ones_like(clip[0])*255) for ax, clip in zip(axes, clips)]
    init = lambda: artists

    anim = animation.FuncAnimation(fig, animate_fn(clips, artists), init_func=init, interval=interval, frames=n_frames)
    return anim



In [ ]:
triplet = next(triplets_segments_gen(1, segments))

In [ ]:
anim = show_clips([get_seg_clip(seg, n_frames=40) for seg in triplet], interval=48)
HTML(anim.to_jshtml())

In [ ]:
anchor_id, pos_id, neg_id = sample_triplets()
print(anchor_id, pos_id, neg_id)

print(labels[anchor_id], labels[pos_id], labels[neg_id])
print(find_segment(anchor_id), find_segment(pos_id), find_segment(neg_id))
(np.linalg.norm(X_encoded[anchor_id] - X_encoded[pos_id]),
np.linalg.norm(X_encoded[anchor_id] - X_encoded[neg_id]),
np.linalg.norm(X_encoded[neg_id] - X_encoded[pos_id]))

In [ ]:
def find_segment(sample_id):
    frame_id = (sample_id + 15) * 4
    for i, seg in enumerate(segments):
        if seg[1] > frame_id:
            return segments[i-1]
        
find_segment(anchor_id), find_segment(pos_id), find_segment(neg_id)

In [ ]:
# def animate_fn(frames, ax):
#     def animate(i):
#         return (ax.imshow(np.ones_like(frames[0])*255),)
#     return animate

# def show_clip(ax, frames):
# #     im = ax.imshow(np.ones_like(frames[0])*255)
#     init = lambda: (ax.imshow(np.ones_like(frames[0])*255),)

#     anim = animation.FuncAnimation(fig, animate_fn(frames, ax), init_func=init, 
#                                    interval=20, frames=len(frames), blit=True)
#     return anim
    
# fig, ax = plt.subplots()
# anim = show_clip(ax, vid[1000:1200:8])

# HTML(anim.to_jshtml())

In [ ]:
def get_clip(idx, fps):
    return vid[idx - 60: idx + 60: 240//fps]


def animate_fn(clips, artists):
    def animate(i):
        for im, clip in zip(artists, clips):
            im.set_data(clip[i])
        return artists
    return animate

def show_clips(clips):
    fig, axes = plt.subplots(ncols=len(clips), figsize=(12, 6))
    n_frames = min(len(clip) for clip in clips)
    for i, ax in enumerate(axes):
        ax.set_title(f"clip {i}")
    artists = [ax.imshow(np.ones_like(clip[0])*255) for ax, clip in zip(axes, clips)]
    init = lambda: artists

    anim = animation.FuncAnimation(fig, animate_fn(clips, artists), init_func=init, interval=20, frames=n_frames)
    return anim

In [ ]:
anchor_id, pos_id, neg_id = next(triplets_gen(1))
print(anchor_id, pos_id, neg_id)

print(labels[anchor_id], labels[pos_id], labels[neg_id])

(np.linalg.norm(X_encoded[anchor_id] - X_encoded[pos_id]),
np.linalg.norm(X_encoded[anchor_id] - X_encoded[neg_id]),
np.linalg.norm(X_encoded[neg_id] - X_encoded[pos_id]))

In [ ]:
def get_clip(idx, fps):
    return vid[4*idx - 60: 4*idx + 60: 240//fps]

def get_clip_segment(idx, fps, max_duration=1):
    segment = find_segment(idx)
    cluster_id, start, length = segment
    mid = start + length // 2
    n_frames = min(length, max_duration * 240)
    return vid[mid - n_frames//2: mid + n_frames//2: 240//fps]
    
anim = show_clips([get_clip_segment(idx, 240) for idx in [anchor_id, pos_id, neg_id]], interval=1000*(1 / 240)*5)


HTML(anim.to_jshtml())

In [ ]:
get_clip(neg_id, 120).shape

In [ ]:
anchor_id = 23420
pos_id, neg_id = get_pos_neg(anchor_id)

anim = show_clip([get_clip(idx, 120) for idx in [anchor_id, pos_id, neg_id]])

HTML(anim.to_jshtml())

In [ ]:
def get_clip(idx, fps):
    return vid[idx - 60: idx + 60: 240//fps]

anim = show_clips([get_clip(idx, 120) for idx in [anchor_id, pos_id, neg_id]])

HTML(anim.to_jshtml())

In [ ]:
def animate_fn(frames, im):
    def animate(i):
        im.set_data(frames[i])
        return (im)
    return animate

def show_clip(frames):
    fig, axes = plt.subplots(ncols=2)

    im1 = axes[0].imshow(np.ones_like(frames[0])*255)
    im2 = axes[1].imshow(np.ones_like(frames[0])*255)
    init = lambda: (im1, im2)

    anim = animation.FuncAnimation(fig, animate_fn(frames, (im1, im2)), init_func=init, interval=20, frames=len(frames))
    return anim
    
anim = show_clip(vid[1000:1200:8])

HTML(anim.to_jshtml())

In [ ]:
HTML(anim.to_jshtml())

In [ ]:
frames = vid[12000:12100:2]

fig, ax = plt.subplots()
im = ax.imshow(np.ones_like(frames[0])*255)

def init():
    im.set_data(np.ones_like(frames[0])*255)
    return (im,)

def animate(i):
    im.set_data(frames[i])
    return (im,)

anim = animation.FuncAnimation(fig, animate, init_func=init, interval=20, frames=len(frames))
HTML(anim.to_jshtml())

In [ ]:
mu, sig = vid.normalized_landmarks.coords.mean(axis=0) , vid.normalized_landmarks.coords.std(axis=0)
normalized_coords = (vid.normalized_landmarks.coords - mu) / (sig + 1e-8)
plt.hist(normalized_coords.flatten(), bins=100, log=True)
1

In [ ]:
idxs = np.where(outliers[:,3,0])[0]
axis = plt.subplot()
plot_coords(axis, vid.normalized_landmarks[idxs[1000]])
Image.fromarray(vid[idxs[1000]])

In [ ]:
normalized_coords[idxs[500]], vid.video.shape, vid.landmarks.body_parts

In [ ]:
idxs = np.where(outliers.sum(axis=(1, 2)) > 0)[0]
Image.fromarray(vid[idxs[500]])

In [ ]:
outliers = np.abs(normalized_coords) > 5
plt.hist(normalized_coords[outliers].flatten(), bins=100, log=True)

In [ ]:
X_sample = X_encoded[np.random.choice(X_encoded.shape[0], size=3000)]
plt.hist(np.linalg.norm(X_sample[np.newaxis] - X_sample[:,np.newaxis], axis=-1).flatten(), bins=100)
0

In [ ]:
X_sample = X_clusters[20]
X_sample = X_sample[np.random.choice(X_sample.shape[0], size=300)]
plt.hist(np.linalg.norm(X_sample[np.newaxis] - X_sample[:,np.newaxis], axis=-1).flatten(), bins=100)

In [ ]:
[np.linalg.norm(X_clusters[i] - kmeans.cluster_centers_[i], axis=1).mean() for i in range(30)]

In [ ]:
plt.hist(np.linalg.norm(np.diff(X_encoded, axis=0), axis=1), bins=100, log=True, density=True)
x_segments1 = X_encoded[np.array([seg[1]//4 for seg in segments])]
x_segments2 = X_encoded[np.array([seg[1]//4 + 1 for seg in segments])]
plt.hist(np.linalg.noclusterrm(x_segments2 - x_segments1, axis=1), bins=100, log=True, alpha=0.5, density=True)


In [ ]:
def plot_coords(ax, coords):
#     ax.scatter(coords[:,1], coords[:,0], c=[np.array(c)/255 for in color_pallete[:-1]])
    for i, bp in enumerate(vid.landmarks.body_parts):
        ax.scatter(coords[i,1], coords[i,0], color=np.array(color_pallete[i])/255, label=bp)
    ax.axis('scaled')
    ax.set_xlim((-100, 100))
    ax.legend(bbox_to_anchor=(1., 1.))
    
    
axis = plt.subplot()
plot_coords(axis, vid.normalized_landmarks[idxs[10]])

In [ ]:
Image.fromarray(frames[3])

In [ ]:
s = slice(100)
s.start is None